In [ ]:
# SFT 코퍼스 클리닝 스크립트 (D1-D3)
#
# [2025-10-31 v6 수정]
# - 'cleaned_text'가 비어있는 버그 수정
# - 'Speeches' 파일의 본문을 통째로 삭제하던
#   탐욕적(greedy) 정규식 `(r"At the .*", "")`를 삭제.
# - 'Speeches' 머리말을 더 안전하게 제거하는 정규식으로 개선.
#
# [동작 방식]
# 1. 'Downloads/statements' 폴더에서 모든 'fomc_*.jsonl' 파일을 로드합니다.
# 2. 모든 데이터를 하나의 pandas DataFrame으로 병합합니다.
# 3. [v4] 날짜(date) 컬럼을 표준 datetime 형식으로 정제 및 NaT 제거.
# 4. URL 기준으로 중복 데이터를 제거합니다.
# 5. [v6] 'text' 컬럼의 텍스트를 정제합니다 (헤더, 꼬리말, \\n 등 노이즈 제거).
# 6. 최종 코퍼스를 'corpus.parquet' 파일로 저장합니다.
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install -r requirements.txt`를 실행하여
#    'pandas', 'pyarrow' 라이브러리를 설치하세요.

import pandas as pd
import glob
import os
import re
from tqdm import tqdm
import datetime

# tqdm이 pandas apply()와 잘 작동하도록 설정
tqdm.pandas()

# --- 1. 설정 (Configuration) ---
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements','data')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "corpus.parquet") # (D1 Deliverable)

# (v3) 텍스트 날짜 파싱을 위한 월(Month) 맵
MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# [v6 수정] 정제(Cleaning)할 노이즈 패턴 (정규식)
CLEANING_REGEX_PAIRS = [
    # [v5] 텍스트로 포함된 줄바꿈 문자 (예: "\\n") 제거
    (r'\\n', ' '), # 텍스트 '\\n' -> 공백
    (r'\\r', ' '), # 텍스트 '\\r' -> 공백
    (r'\\t', ' '), # 텍스트 '\\t' -> 공백
    
    # --- [v6] Speeches 머리말 정제 (안전한 방식) ---
    # 예: "...Kansas City, Missouri Share Watch Live ... Good morning"
    #     -> "Good morning"
    (r".*Share Watch Live.*?Good morning", "Good morning"),
    (r".*Share Watch Live.*?Good afternoon", "Good afternoon"),
    (r".*Share Watch Live.*?Good evening", "Good evening"),
    # 'Watch Live'가 없는 구형 Speeches
    # 예: "...Washington, D.C. Thank you"
    (r"^At the .*?Thank you\. ", "Thank you. "), # 'At the'로 시작하는 경우
    
    # --- 웹사이트 공통 노이즈 ---
    (r"Home\s*\|", ""),
    (r"News & Events\s*\|", ""),
    (r"Monetary Policy\s*\|", ""),
    (r"About the Fed\s*\|", ""),
    (r"Board of Governors of the Federal Reserve System", ""),
    (r"Federal Open Market Committee", ""),
    (r"Skip to main content", ""),
    (r"Last Update:.*", ""),
    (r"An official website of the United States Government", ""),
    (r"Here's how you know", ""),
    (r"Search\s*Submit Search Button", ""),
    (r"Back to Top", ""),
    (r"Stay Connected", ""),
    (r"Tools and Information", ""),
    (r"Contact\s*\|\s*Publications\s*\|", ""),
    (r"Freedom of Information \(FOIA\)", ""),
    (r"Accessibility", ""),
    (r"Privacy Program", ""),
    (r"Website Policies", ""),
    (r"Español", ""),
    (r"Office of Inspector General", ""),
    (r"Budget & Performance", ""),
    (r"No FEAR Act", ""),
    (r"Link to USA\.gov", ""),
    (r"Link to Open\.gov", ""),
    (r"\(PDF\)", ""),
    (r"\(HTML\)", ""),
    (r"Watch Live", ""), # 'Share Watch Live'에서 놓친 나머지
    (r"Implementation Note", ""),
    (r"Release Date:.*", ""),
    (r"For immediate release", ""),
    (r"FRB: Press Release --.*", ""),
    # --- Minutes/Speeches에서 발견되는 패턴 ---
    (r"Press Conference", ""),
    (r"Projection Materials", ""),
    (r"\(Released.*\)", ""),
    (r"Listen", ""),
    # (r"At the .*", ""), # [v5 버그] -> [v6]에서 삭제됨
    (r"via prerecorded video", ""),
    (r"\(virtual\)", ""),
    (r"\(via satellite\)", ""),
    # --- 불필요한 공백 정제 ---
    (r'\s+', ' '), # 모든 실제 공백/줄바꿈을 1칸 공백으로
]

# --- 2. 헬퍼 함수 ---

def clean_text(text):
    """정규식 패턴을 적용하여 텍스트를 정제합니다."""
    if not isinstance(text, str):
        return ""
        
    cleaned = text
    for pattern, replacement in compiled_regex:
        cleaned = pattern.sub(replacement, cleaned)
        
    return cleaned.strip()

# [v3] 정규식 패턴 미리 컴파일
compiled_regex = []
for pattern_str, replacement_str in CLEANING_REGEX_PAIRS:
    try:
        compiled_regex.append((re.compile(pattern_str, re.IGNORECASE), replacement_str))
    except re.error as e:
        print(f"[경고] 정규식 컴파일 오류: '{pattern_str}' ({e})")

# [v3 추가] 날짜 정제 함수 (D1)
def parse_messy_date(date_obj):
    """
    "2019-01-30" (정상)과 "February 1-2 Meeting - 2000" (텍스트)를
    모두 datetime 객체로 변환합니다.
    """
    if pd.isna(date_obj):
        return None
        
    # 1. 'speeches', 'minutes' 파일처럼 이미 날짜 형식인 경우
    try:
        parsed_date = pd.to_datetime(date_obj, errors='coerce')
        if not pd.isna(parsed_date):
            return parsed_date
    except Exception:
        pass # 실패하면 2단계로

    # 2. 'statements' 파일의 텍스트 형식 ("February 1-2 Meeting - 2000")
    try:
        date_str = str(date_obj)
        
        month_day_match = re.search(r'([A-Za-z]+)\s+([\d-]+)', date_str)
        year_match = re.search(r'(\d{4})', date_str)
        
        if month_day_match and year_match:
            month_str = month_day_match.group(1)
            day_str = month_day_match.group(2)
            year_int = int(year_match.group(1))
            
            month_int = MONTH_MAP.get(month_str)
            day_int = int(re.split(r'[-/]', day_str)[-1]) 
            
            if month_int:
                return datetime.datetime(year_int, month_int, day_int)
                
    except Exception as e:
        pass

    return None # 모든 파싱 실패

# --- 3. 메인 실행 ---
if __name__ == "__main__":
    
    # --- 1. 데이터 로드 (D1) ---
    jsonl_files = glob.glob(os.path.join(DOWNLOADS_DIR, "fomc_*.jsonl"))
    
    if not jsonl_files:
        print(f"[오류] '{DOWNLOADS_DIR}'에서 'fomc_*.jsonl' 파일을 찾을 수 없습니다.")
        print("스크립트를 종료합니다.")
    else:
        print(f"--- SFT 코퍼스 클리닝 시작 (D1-D3) (v6) ---") # v6
        print(f"총 {len(jsonl_files)}개의 .jsonl 파일을 찾았습니다:")
        for f in jsonl_files:
            print(f"  - {os.path.basename(f)}")
        
        df_list = []
        for file_path in jsonl_files:
            try:
                df_part = pd.read_json(file_path, lines=True)
                df_part['source_file'] = os.path.basename(file_path)
                df_list.append(df_part)
            except Exception as e:
                print(f"  [경고] '{file_path}' 파일 로드 실패: {e}")

        if not df_list:
            print("[오류] 로드할 수 있는 데이터가 없습니다. 스크립트를 종료합니다.")
            exit()
            
        df = pd.concat(df_list, ignore_index=True)
        print(f"\n--- 1. 로드 완료 ---")
        print(f"총 {len(df)}개의 원본 문서를 로드했습니다.")
        
        # --- 2. [v4] 날짜 정제 (D1) ---
        print(f"--- 2. 날짜(date) 컬럼 표준화 시작 ---")
        df['datetime_clean'] = df['date'].progress_apply(parse_messy_date)
        
        df['datetime_clean'] = pd.to_datetime(df['datetime_clean'], errors='coerce')
        
        original_count_date = len(df)
        df = df.dropna(subset=['datetime_clean'])
        print(f"날짜 파싱 실패/누락 행 {original_count_date - len(df)}개를 삭제했습니다.")
        
        df['year_clean'] = df['datetime_clean'].dt.year

        # --- 3. 중복 제거 (D1) ---
        df = df.dropna(subset=['url']).drop_duplicates(subset=['url'])
        df = df.dropna(subset=['text']).drop_duplicates(subset=['text'])
        
        print(f"--- 3. 중복 제거 완료 ---")
        print(f"중복 제거 후 {len(df)}개의 고유 문서를 확보했습니다.")

        # --- 4. 텍스트 클리닝 (D1) ---
        print(f"--- 4. 텍스트 정제 시작 ---")
        if 'text' not in df.columns:
            df['text'] = ""
        df['text'] = df['text'].fillna("")
        
        df['cleaned_text'] = df['text'].progress_apply(clean_text)
        
        original_count_text = len(df)
        # [v6] 정제 후 100글자 미만의 너무 짧은 텍스트도 노이즈로 간주하고 제거
        df = df[df['cleaned_text'].str.len() > 100]
        print(f"정제 후 비어있거나 너무 짧은(100자 미만) 행 {original_count_text - len(df)}개를 삭제했습니다.")

        # --- 5. 최종 저장 (D1-D3) ---
        final_columns = ['datetime_clean', 'year_clean', 'source_type', 'speaker', 'title', 'cleaned_text', 'url', 'source_file']
        
        for col in final_columns:
            if col not in df.columns:
                df[col] = None
        
        df_final = df[final_columns].rename(columns={
            "datetime_clean": "date",
            "year_clean": "year"
        })
        
        df_final = df_final.sort_values(by='date')
        
        df_final.to_parquet(OUTPUT_FILE, index=False)
        
        print(f"\n--- 5. 저장 완료 ---")
        print(f"최종 정제된 코퍼스(corpus)를 '{OUTPUT_FILE}' 파일에 저장했습니다.")
        print(f"최종 문서 수: {len(df_final)}")



--- SFT 코퍼스 클리닝 시작 (D1-D3) (v6) ---
총 4개의 .jsonl 파일을 찾았습니다:
  - fomc_minutes_2000-2019.jsonl
  - fomc_minutes_2020-present.jsonl
  - fomc_speeches_2000-present.jsonl
  - fomc_statements_2000-present.jsonl

--- 1. 로드 완료 ---
총 1651개의 원본 문서를 로드했습니다.
--- 2. 날짜(date) 컬럼 표준화 시작 ---


100%|██████████| 1651/1651 [00:00<00:00, 10094.01it/s]


날짜 파싱 실패/누락 행 4개를 삭제했습니다.
--- 3. 중복 제거 완료 ---
중복 제거 후 1631개의 고유 문서를 확보했습니다.
--- 4. 텍스트 정제 시작 ---


  0%|          | 5/1631 [12:38<64:13:33, 142.20s/it]

In [ ]:
# SFT 코퍼스 클리닝 스크립트 (D1-D3)
#
# [2025-10-31 v7 수정]
# - 14분에 5개 처리되는 "catastrophic backtracking" (정규식 성능 저하) 버그 수정
# - CLEANING_REGEX를 'COMMON' (빠름)과 'SPEECH_HEADER' (느림)로 분리
# - clean_text 함수가 'source_type'을 확인하여, 'Speech'가 아닐 경우
#   느린 헤더 정규식을 건너뛰도록 수정
# - .progress_apply(axis=1)을 사용하여 행(row) 단위로 적용
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install -r requirements.txt`를 실행하여
#    'pandas', 'pyarrow' 라이브러리를 설치하세요.

import pandas as pd
import glob
import os
import re
from tqdm import tqdm
import datetime

# tqdm이 pandas apply()와 잘 작동하도록 설정
tqdm.pandas()

# --- 1. 설정 (Configuration) ---
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements','Data')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "corpus.parquet") # (D1 Deliverable)

MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# [v7 수정] (1) 공통 정규식 (빠름, 모든 텍스트에 적용)
COMMON_REGEX_PAIRS = [
    (r'\\n', ' '), (r'\\r', ' '), (r'\\t', ' '),
    (r"Home\s*\|", ""), (r"News & Events\s*\|", ""), (r"Monetary Policy\s*\|", ""),
    (r"About the Fed\s*\|", ""), (r"Board of Governors of the Federal Reserve System", ""),
    (r"Federal Open Market Committee", ""), (r"Skip to main content", ""),
    (r"Last Update:.*", ""), (r"An official website of the United States Government", ""),
    (r"Here's how you know", ""), (r"Search\s*Submit Search Button", ""),
    (r"Back to Top", ""), (r"Stay Connected", ""), (r"Tools and Information", ""),
    (r"Contact\s*\|\s*Publications\s*\|", ""), (r"Freedom of Information \(FOIA\)", ""),
    (r"Accessibility", ""), (r"Privacy Program", ""), (r"Website Policies", ""),
    (r"Español", ""), (r"Office of Inspector General", ""), (r"Budget & Performance", ""),
    (r"No FEAR Act", ""), (r"Link to USA\.gov", ""), (r"Link to Open\.gov", ""),
    (r"\(PDF\)", ""), (r"\(HTML\)", ""), (r"Watch Live", ""), (r"Implementation Note", ""),
    (r"Release Date:.*", ""), (r"For immediate release", ""), (r"FRB: Press Release --.*", ""),
    (r"Press Conference", ""), (r"Projection Materials", ""), (r"\(Released.*\)", ""),
    (r"Listen", ""), (r"via prerecorded video", ""), (r"\(virtual\)", ""), (r"\(via satellite\)", ""),
    (r'\s+', ' '), # 맨 마지막에 공백 정제
]

# [v7 수정] (2) 연설문 헤더 정규식 (느림, 'Speeches'에만 적용)
SPEECH_HEADER_REGEX_PAIRS = [
    (r".*Share Watch Live.*?Good morning", "Good morning"),
    (r".*Share Watch Live.*?Good afternoon", "Good afternoon"),
    (r".*Share Watch Live.*?Good evening", "Good evening"),
    (r"^At the .*?Thank you\. ", "Thank you. "),
]

# --- 2. 헬퍼 함수 ---

# [v7 수정] clean_text 함수가 'source_type'을 인자로 받음
def clean_text(text, source_type):
    """정규식 패턴을 적용하여 텍스트를 정제합니다."""
    if not isinstance(text, str):
        return ""
        
    cleaned = text
    
    # 1. 모든 문서에 공통 정규식 적용
    for pattern, replacement in compiled_common_regex:
        cleaned = pattern.sub(replacement, cleaned)
        
    # 2. [v7] 'Speech'인 경우에만 느린 헤더 정규식 추가 적용
    if 'speech' in str(source_type).lower():
        for pattern, replacement in compiled_speech_header_regex:
            cleaned = pattern.sub(replacement, cleaned)
            
    return cleaned.strip()

# [v7 수정] 정규식 2개 리스트로 컴파일
compiled_common_regex = []
for pattern_str, replacement_str in COMMON_REGEX_PAIRS:
    try:
        compiled_common_regex.append((re.compile(pattern_str, re.IGNORECASE), replacement_str))
    except re.error as e:
        print(f"[경고] 공통 정규식 컴파일 오류: '{pattern_str}' ({e})")

compiled_speech_header_regex = []
for pattern_str, replacement_str in SPEECH_HEADER_REGEX_PAIRS:
    try:
        compiled_speech_header_regex.append((re.compile(pattern_str, re.IGNORECASE), replacement_str))
    except re.error as e:
        print(f"[경고] 연설문 헤더 정규식 컴파일 오류: '{pattern_str}' ({e})")

def parse_messy_date(date_obj):
    """
    "2019-01-30" (정상)과 "February 1-2 Meeting - 2000" (텍스트)를
    모두 datetime 객체로 변환합니다.
    """
    if pd.isna(date_obj):
        return None
        
    # 1. 'speeches', 'minutes' 파일처럼 이미 날짜 형식인 경우
    try:
        parsed_date = pd.to_datetime(date_obj, errors='coerce')
        if not pd.isna(parsed_date):
            return parsed_date
    except Exception:
        pass # 실패하면 2단계로

    # 2. 'statements' 파일의 텍스트 형식 ("February 1-2 Meeting - 2000")
    try:
        date_str = str(date_obj)
        
        month_day_match = re.search(r'([A-Za-z]+)\s+([\d-]+)', date_str)
        year_match = re.search(r'(\d{4})', date_str)
        
        if month_day_match and year_match:
            month_str = month_day_match.group(1)
            day_str = month_day_match.group(2)
            year_int = int(year_match.group(1))
            
            month_int = MONTH_MAP.get(month_str)
            day_int = int(re.split(r'[-/]', day_str)[-1]) 
            
            if month_int:
                return datetime.datetime(year_int, month_int, day_int)
                
    except Exception as e:
        pass

    return None # 모든 파싱 실패

# --- 3. 메인 실행 ---
if __name__ == "__main__":
    
    # --- 1. 데이터 로드 (D1) ---
    jsonl_files = glob.glob(os.path.join(DOWNLOADS_DIR, "fomc_*.jsonl"))
    
    if not jsonl_files:
        print(f"[오류] '{DOWNLOADS_DIR}'에서 'fomc_*.jsonl' 파일을 찾을 수 없습니다.")
        print("스크립트를 종료합니다.")
    else:
        print(f"--- SFT 코퍼스 클리닝 시작 (D1-D3) (v7) ---") # v7
        print(f"총 {len(jsonl_files)}개의 .jsonl 파일을 찾았습니다:")
        for f in jsonl_files:
            print(f"  - {os.path.basename(f)}")
        
        df_list = []
        for file_path in jsonl_files:
            try:
                df_part = pd.read_json(file_path, lines=True)
                df_part['source_file'] = os.path.basename(file_path)
                df_list.append(df_part)
            except Exception as e:
                print(f"  [경고] '{file_path}' 파일 로드 실패: {e}")

        if not df_list:
            print("[오류] 로드할 수 있는 데이터가 없습니다. 스크립트를 종료합니다.")
            exit()
            
        df = pd.concat(df_list, ignore_index=True)
        print(f"\n--- 1. 로드 완료 ---")
        print(f"총 {len(df)}개의 원본 문서를 로드했습니다.")
        
        # --- 2. 날짜 정제 (D1) ---
        print(f"--- 2. 날짜(date) 컬럼 표준화 시작 ---")
        df['datetime_clean'] = df['date'].progress_apply(parse_messy_date)
        
        df['datetime_clean'] = pd.to_datetime(df['datetime_clean'], errors='coerce')
        
        original_count_date = len(df)
        df = df.dropna(subset=['datetime_clean'])
        print(f"날짜 파싱 실패/누락 행 {original_count_date - len(df)}개를 삭제했습니다.")
        
        df['year_clean'] = df['datetime_clean'].dt.year

        # --- 3. 중복 제거 (D1) ---
        df = df.dropna(subset=['url']).drop_duplicates(subset=['url'])
        df = df.dropna(subset=['text']).drop_duplicates(subset=['text'])
        
        print(f"--- 3. 중복 제거 완료 ---")
        print(f"중복 제거 후 {len(df)}개의 고유 문서를 확보했습니다.")

        # --- 4. 텍스트 클리닝 (D1) ---
        print(f"--- 4. 텍스트 정제 시작 ---")
        # [v7] 'source_type'도 .fillna()
        if 'text' not in df.columns: df['text'] = ""
        if 'source_type' not in df.columns: df['source_type'] = ""
        df['text'] = df['text'].fillna("")
        df['source_type'] = df['source_type'].fillna("")
        
        # [v7 수정] .progress_apply()에서 lambda 함수로 source_type 전달 (axis=1)
        print("정제 작업 중... (v7은 v6보다 다소 느릴 수 있으나, 멈춘 것이 아닙니다.)")
        df['cleaned_text'] = df.progress_apply(
            lambda row: clean_text(row['text'], row['source_type']),
            axis=1 # 행(row) 단위로 적용
        )
        
        original_count_text = len(df)
        df = df[df['cleaned_text'].str.len() > 100] # 100자 미만 제거
        print(f"정제 후 비어있거나 너무 짧은(100자 미만) 행 {original_count_text - len(df)}개를 삭제했습니다.")

        # --- 5. 최종 저장 (D1-D3) ---
        final_columns = ['datetime_clean', 'year_clean', 'source_type', 'speaker', 'title', 'cleaned_text', 'url', 'source_file']
        
        for col in final_columns:
            if col not in df.columns:
                df[col] = None
        
        df_final = df[final_columns].rename(columns={
            "datetime_clean": "date",
            "year_clean": "year"
        })
        
        df_final = df_final.sort_values(by='date')
        
        df_final.to_parquet(OUTPUT_FILE, index=False)
        
        print(f"\n--- 5. 저장 완료 ---")
        print(f"최종 정제된 코퍼스(corpus)를 '{OUTPUT_FILE}' 파일에 저장했습니다.")
        print(f"최종 문서 수: {len(df_final)}")


--- SFT 코퍼스 클리닝 시작 (D1-D3) (v7) ---
총 4개의 .jsonl 파일을 찾았습니다:
  - fomc_minutes_2000-2019.jsonl
  - fomc_minutes_2020-present.jsonl
  - fomc_speeches_2000-present.jsonl
  - fomc_statements_2000-present.jsonl

--- 1. 로드 완료 ---
총 1651개의 원본 문서를 로드했습니다.
--- 2. 날짜(date) 컬럼 표준화 시작 ---


100%|██████████| 1651/1651 [00:00<00:00, 10890.84it/s]


날짜 파싱 실패/누락 행 4개를 삭제했습니다.
--- 3. 중복 제거 완료 ---
중복 제거 후 1631개의 고유 문서를 확보했습니다.
--- 4. 텍스트 정제 시작 ---
정제 작업 중... (v7은 v6보다 다소 느릴 수 있으나, 멈춘 것이 아닙니다.)


 50%|████▉     | 811/1631 [2:30:01<5:00:24, 21.98s/it] 

더 빠른 버전 이거 한번 돌려보기 !!

In [ ]:
# -*- coding: utf-8 -*-
"""
SFT 코퍼스 클리닝 스크립트 (D1-D3)  [2025-10-31 v7-fast]
- v7의 기능 유지 + 속도 최적화 (행 단위 apply 제거, 벡터화 클리닝)
- 'Speeches'에만 느린 헤더 정규식 적용 (부분 시리즈 선택 + 트리거 프리필터)
- 날짜 파싱은 안전성을 위해 progress_apply 유지

[실행 전]
pip install -r requirements.txt   # pandas, pyarrow, tqdm
"""

import os
import re
import glob
import datetime
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# --- 1) 설정 ---
#DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements', 'Data')
DOWNLOADS_DIR = r"C:\Users\jeong\Downloads\statements\statements\Data"
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "corpus.parquet")  # (D1 Deliverable)

MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# --- 2) 정규식 패턴 정의 ---
# [공통: 빠름] (모든 소스에 적용)
COMMON_REGEX_PAIRS = [
    (r'\\n', ' '), (r'\\r', ' '), (r'\\t', ' '),
    (r"Home\s*\|", ""), (r"News & Events\s*\|", ""), (r"Monetary Policy\s*\|", ""),
    (r"About the Fed\s*\|", ""), (r"Board of Governors of the Federal Reserve System", ""),
    (r"Federal Open Market Committee", ""), (r"Skip to main content", ""),
    (r"Last Update:.*", ""), (r"An official website of the United States Government", ""),
    (r"Here's how you know", ""), (r"Search\s*Submit Search Button", ""),
    (r"Back to Top", ""), (r"Stay Connected", ""), (r"Tools and Information", ""),
    (r"Contact\s*\|\s*Publications\s*\|", ""), (r"Freedom of Information \(FOIA\)", ""),
    (r"Accessibility", ""), (r"Privacy Program", ""), (r"Website Policies", ""),
    (r"Español", ""), (r"Office of Inspector General", ""), (r"Budget & Performance", ""),
    (r"No FEAR Act", ""), (r"Link to USA\.gov", ""), (r"Link to Open\.gov", ""),
    (r"\(PDF\)", ""), (r"\(HTML\)", ""), (r"Watch Live", ""), (r"Implementation Note", ""),
    (r"Release Date:.*", ""), (r"For immediate release", ""), (r"FRB: Press Release --.*", ""),
    (r"Press Conference", ""), (r"Projection Materials", ""), (r"\(Released.*\)", ""),
    (r"Listen", ""), (r"via prerecorded video", ""), (r"\(virtual\)", ""), (r"\(via satellite\)", ""),
    (r'\s+', ' ')  # 공백 정리 (추가로 마지막에 한 번 더 정리함)
]

# [스피치 헤더: 느림] (Speeches에만 적용)
SPEECH_HEADER_REGEX_PAIRS = [
    (r".*Share Watch Live.*?Good morning", "Good morning"),
    (r".*Share Watch Live.*?Good afternoon", "Good afternoon"),
    (r".*Share Watch Live.*?Good evening", "Good evening"),
    (r"^At the .*?Thank you\. ", "Thank you. "),
]

# --- 3) 정규식 컴파일 ---
compiled_common_regex = []
for pattern_str, replacement_str in COMMON_REGEX_PAIRS:
    try:
        compiled_common_regex.append((re.compile(pattern_str, re.IGNORECASE), replacement_str))
    except re.error as e:
        print(f"[경고] 공통 정규식 컴파일 오류: '{pattern_str}' ({e})")

compiled_speech_header_regex = []
for pattern_str, replacement_str in SPEECH_HEADER_REGEX_PAIRS:
    try:
        compiled_speech_header_regex.append((re.compile(pattern_str, re.IGNORECASE), replacement_str))
    except re.error as e:
        print(f"[경고] 연설문 헤더 정규식 컴파일 오류: '{pattern_str}' ({e})")


# --- 4) 날짜 파싱 ---
def parse_messy_date(date_obj):
    """
    '2019-01-30' 같은 ISO 날짜와
    'February 1-2 Meeting - 2000' 같은 텍스트 모두를 datetime으로 변환.
    """
    if pd.isna(date_obj):
        return None

    # 1) 이미 날짜 포맷이면 우선 시도
    try:
        parsed_date = pd.to_datetime(date_obj, errors='coerce')
        if not pd.isna(parsed_date):
            return parsed_date
    except Exception:
        pass  # 실패 시 2단계

    # 2) 텍스트 포맷 해석
    try:
        date_str = str(date_obj)

        month_day_match = re.search(r'([A-Za-z]+)\s+([\d-]+)', date_str)
        year_match = re.search(r'(\d{4})', date_str)

        if month_day_match and year_match:
            month_str = month_day_match.group(1)
            day_str = month_day_match.group(2)
            year_int = int(year_match.group(1))

            month_int = MONTH_MAP.get(month_str)
            # "1-2" 같은 범위면 마지막 숫자를 사용
            day_int = int(re.split(r'[-/]', day_str)[-1])

            if month_int:
                return datetime.datetime(year_int, month_int, day_int)
    except Exception:
        pass

    return None  # 모두 실패


# --- 5) 텍스트 클리닝 (벡터화 버전) ---
def clean_text_vectorized(df: pd.DataFrame) -> pd.Series:
    """
    행 단위 apply(axis=1)를 쓰지 않고,
    시리즈 단위로 공통 정규식 → (스피치만) 헤더 정규식 순서로 적용.
    """
    # 안전한 기본 컬럼
    if 'text' not in df.columns:
        df['text'] = ""
    if 'source_type' not in df.columns:
        df['source_type'] = ""

    s = df['text'].fillna("").astype(str)
    source_type = df['source_type'].fillna("")

    # 1) 값싼 전처리: 역슬래시 포함 제어문자 치환 (정규식 사용 안 함)
    #    원문이 실제 개행(\n)일 수도, '역슬래시+n' 문자열일 수도 있으므로 둘 다 정리
    s = (s.str.replace('\\n', ' ', regex=False)
           .str.replace('\\r', ' ', regex=False)
           .str.replace('\\t', ' ', regex=False)
           .str.replace('\n', ' ', regex=False)
           .str.replace('\r', ' ', regex=False)
           .str.replace('\t', ' ', regex=False))

    # 2) 공통 정규식 일괄 적용
    for pattern, repl in compiled_common_regex:
        s = s.str.replace(pattern, repl, regex=True)

    # 3) 스피치만 헤더 정규식 적용 (부분 시리즈 선택 + 프리필터)
    mask_speech = source_type.str.contains('speech', case=False, na=False)

    # 헤더 트리거(없으면 아예 정규식 skip)
    header_trigger = (
        s.str.contains('Good morning|Good afternoon|Good evening', case=False, na=False) |
        s.str.contains('Share Watch Live', case=False, na=False) |
        s.str.match(r'(?i)At the .*?Thank you\.', na=False)
    )
    mask_target = mask_speech & header_trigger

    if mask_target.any():
        s_sub = s[mask_target]
        for pattern, repl in compiled_speech_header_regex:
            s_sub = s_sub.str.replace(pattern, repl, regex=True)
        s.loc[mask_target] = s_sub

    # 4) 마지막 공백 정리 1회
    s = s.str.replace(r'\s+', ' ', regex=True).str.strip()

    return s


# --- 6) 메인 ---
if __name__ == "__main__":
    jsonl_files = glob.glob(os.path.join(DOWNLOADS_DIR, "fomc_*.jsonl"))

    if not jsonl_files:
        print(f"[오류] '{DOWNLOADS_DIR}'에서 'fomc_*.jsonl' 파일을 찾을 수 없습니다.")
        raise SystemExit(1)

    print("--- SFT 코퍼스 클리닝 시작 (D1-D3) (v7-fast) ---")
    print(f"총 {len(jsonl_files)}개의 .jsonl 파일을 찾았습니다:")
    for f in jsonl_files:
        print(f"  - {os.path.basename(f)}")

    # 1) 로드
    df_list = []
    for file_path in jsonl_files:
        try:
            df_part = pd.read_json(file_path, lines=True)
            df_part['source_file'] = os.path.basename(file_path)
            df_list.append(df_part)
        except Exception as e:
            print(f"  [경고] '{file_path}' 파일 로드 실패: {e}")

    if not df_list:
        print("[오류] 로드할 수 있는 데이터가 없습니다. 스크립트를 종료합니다.")
        raise SystemExit(1)

    df = pd.concat(df_list, ignore_index=True)
    print("\n--- 1. 로드 완료 ---")
    print(f"총 {len(df)}개의 원본 문서를 로드했습니다.")

    # 2) 날짜 정제
    print("--- 2. 날짜(date) 컬럼 표준화 시작 ---")
    if 'date' not in df.columns:
        df['date'] = None

    df['datetime_clean'] = df['date'].progress_apply(parse_messy_date)
    df['datetime_clean'] = pd.to_datetime(df['datetime_clean'], errors='coerce')

    original_count_date = len(df)
    df = df.dropna(subset=['datetime_clean'])
    print(f"날짜 파싱 실패/누락 행 {original_count_date - len(df)}개를 삭제했습니다.")

    df['year_clean'] = df['datetime_clean'].dt.year

    # 3) 중복 제거
    print("--- 3. 중복 제거 ---")
    if 'url' not in df.columns:
        df['url'] = None
    if 'text' not in df.columns:
        df['text'] = ""

    before = len(df)
    df = df.dropna(subset=['url']).drop_duplicates(subset=['url'])
    df = df.dropna(subset=['text']).drop_duplicates(subset=['text'])
    print(f"중복 제거 후 {before} → {len(df)}개")

    # 4) 텍스트 클리닝 (벡터화)
    print("--- 4. 텍스트 정제 시작 (벡터화) ---")
    df['cleaned_text'] = clean_text_vectorized(df)

    original_count_text = len(df)
    df = df[df['cleaned_text'].str.len() > 100]  # 100자 미만 제거
    print(f"정제 후 비어있거나 너무 짧은(100자 미만) 행 {original_count_text - len(df)}개를 삭제했습니다.")

    # 5) 최종 저장
    print("--- 5. 저장 ---")
    final_columns = ['datetime_clean', 'year_clean', 'source_type', 'speaker',
                     'title', 'cleaned_text', 'url', 'source_file']

    for col in final_columns:
        if col not in df.columns:
            df[col] = None

    df_final = df[final_columns].rename(columns={
        "datetime_clean": "date",
        "year_clean": "year"
    }).sort_values(by='date')

    # .parquet로 저장 (pyarrow 필요)
    df_final.to_parquet(OUTPUT_FILE, index=False)
    print(f"\n--- 완료 ---")
    print(f"최종 정제된 코퍼스를 '{OUTPUT_FILE}'에 저장했습니다.")
    print(f"최종 문서 수: {len(df_final)}")


--- SFT 코퍼스 클리닝 시작 (D1-D3) (v7-fast) ---
총 4개의 .jsonl 파일을 찾았습니다:
  - fomc_minutes_2000-2019.jsonl
  - fomc_minutes_2020-present.jsonl
  - fomc_speeches_2000-present.jsonl
  - fomc_statements_2000-present.jsonl

--- 1. 로드 완료 ---
총 1651개의 원본 문서를 로드했습니다.
--- 2. 날짜(date) 컬럼 표준화 시작 ---


100%|██████████| 1651/1651 [00:00<00:00, 52101.78it/s]

날짜 파싱 실패/누락 행 4개를 삭제했습니다.
--- 3. 중복 제거 ---
중복 제거 후 1647 → 1631개
--- 4. 텍스트 정제 시작 (벡터화) ---


In [5]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\jeong\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.0 MB 6.9 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.0 MB 6.9 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 6.9 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/11.0 MB 6.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.0 MB 6.3 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 6.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.9/11.0 MB 6.2 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 6.2 MB/s  0:00:01
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.8 MB 7.5 MB/s eta 0:00:02
   ------- --


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\jeong\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
# corpus.parquet 파일의 내용을 확인하기 위한 스크립트
#
# [동작 방식]
# 1. 'corpus.parquet' 파일을 pandas DataFrame으로 로드합니다.
# 2. df.info() - 컬럼별 타입, 누락된 값(non-null) 개수를 출력합니다.
# 3. df.head() - 상위 5개 행을 출력합니다.
# 4. df['source_type'].value_counts() - 수집된 데이터 소스별 개수를 출력합니다.
# 5. 정제된 텍스트 샘플 3개를 출력합니다.
#
# [중요] 실행 전 설정:
# 1. 'pandas', 'pyarrow'가 설치되어 있어야 합니다.
#    (clean_corpus.py를 실행했다면 이미 설치되어 있습니다.)

import pandas as pd
import os

# --- 1. 설정 (Configuration) ---
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements','Data')
CORPUS_FILE = os.path.join(DOWNLOADS_DIR, "corpus.parquet")

# --- 2. 메인 실행 ---
if __name__ == "__main__":
    
    if not os.path.exists(CORPUS_FILE):
        print(f"[오류] '{CORPUS_FILE}'을 찾을 수 없습니다.")
        print("먼저 `clean_corpus.py` 스크립트를 실행하여 'corpus.parquet' 파일을 생성해야 합니다.")
    else:
        print(f"--- 'corpus.parquet' 파일 분석 시작 ---")
        
        # 1. Parquet 파일 로드
        df = pd.read_parquet(CORPUS_FILE)
        
        print(f"\n[ 1. DataFrame 정보 (df.info()) ]")
        # .info()는 컬럼명, 데이터 개수, 데이터 타입을 보여줍니다.
        df.info()
        
        print(f"\n\n[ 2. 데이터 상위 5개 (df.head()) ]")
        # .head()는 상위 5개 행을 보여줍니다.
        # (date, year, speaker, title 등이 잘 파싱되었는지 확인)
        print(df.head())
        
        print(f"\n\n[ 3. 데이터 소스별 개수 (value_counts()) ]")
        # (D1) 데이터가 균형있게 수집되었는지 확인
        if 'source_type' in df.columns:
            print(df['source_type'].value_counts())
        else:
            print("'source_type' 컬럼을 찾을 수 없습니다.")
            
        print(f"\n\n[ 4. 정제된 텍스트(cleaned_text) 샘플 3개 ]")
        # (D1) 텍스트가 깨끗하게 정제되었는지 (꼬리말/메뉴 제거) 확인
        if 'cleaned_text' in df.columns and len(df) >= 3:
            pd.set_option('display.max_colwidth', 500) # 텍스트가 잘리지 않게 설정
            print(df.sample(n=3)['cleaned_text'])
        else:
            print("'cleaned_text' 컬럼이 없거나 데이터가 3개 미만입니다.")
            
        print(f"\n--- 분석 완료 ---")

--- 'corpus.parquet' 파일 분석 시작 ---

[ 1. DataFrame 정보 (df.info()) ]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1631 non-null   datetime64[ns]
 1   year          1631 non-null   int64         
 2   source_type   1631 non-null   object        
 3   speaker       1263 non-null   object        
 4   title         1263 non-null   object        
 5   cleaned_text  1631 non-null   object        
 6   url           1631 non-null   object        
 7   source_file   1631 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 102.1+ KB


[ 2. 데이터 상위 5개 (df.head()) ]
        date  year                source_type speaker title  \
0 2000-02-02  2000             FOMC Statement    None  None   
1 2000-02-02  2000  FOMC Minutes (Historical)    None  None   
2 2000-03-21  2000  FOMC Minutes (Hi

In [13]:
# corpus.parquet 파일의 내용을 확인하기 위한 스크립트
#
# [동작 방식]
# 1. 'corpus.parquet' 파일을 pandas DataFrame으로 로드합니다.
# 2. df.info() - 컬럼별 타입, 누락된 값(non-null) 개수를 출력합니다.
# 3. df.head() - 상위 5개 행을 출력합니다.
# 4. df['source_type'].value_counts() - 수집된 데이터 소스별 개수를 출력합니다.
# 5. 정제된 텍스트 샘플 3개를 출력합니다.
#
# [중요] 실행 전 설정:
# 1. 'pandas', 'pyarrow'가 설치되어 있어야 합니다.
#    (clean_corpus.py를 실행했다면 이미 설치되어 있습니다.)

import pandas as pd
import os

# --- 1. 설정 (Configuration) ---
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements','Data')
CORPUS_FILE = os.path.join(DOWNLOADS_DIR, "corpus.parquet")

# --- 2. 메인 실행 ---
if __name__ == "__main__":
    
    if not os.path.exists(CORPUS_FILE):
        print(f"[오류] '{CORPUS_FILE}'을 찾을 수 없습니다.")
        print("먼저 `clean_corpus.py` 스크립트를 실행하여 'corpus.parquet' 파일을 생성해야 합니다.")
    else:
        print(f"--- 'corpus.parquet' 파일 분석 시작 ---")
        
        # 1. Parquet 파일 로드
        df = pd.read_parquet(CORPUS_FILE)
        
        # print(f"\n[ 1. DataFrame 정보 (df.info()) ]")
        # .info()는 컬럼명, 데이터 개수, 데이터 타입을 보여줍니다.
        # df.info()
        
        # print(f"\n\n[ 2. 데이터 상위 5개 (df.head()) ]")
        # .head()는 상위 5개 행을 보여줍니다.
        # (date, year, speaker, title 등이 잘 파싱되었는지 확인)
        # print(df.head())
        
        # print(f"\n\n[ 3. 데이터 소스별 개수 (value_counts()) ]")
        # (D1) 데이터가 균형있게 수집되었는지 확인
        # if 'source_type' in df.columns:
        #     print(df['source_type'].value_counts())
        # else:
        #     print("'source_type' 컬럼을 찾을 수 없습니다.")
            
        print(f"\n\n[ 4. 정제된 텍스트(cleaned_text) 샘플 20개 ]")
        # (D1) 텍스트가 깨끗하게 정제되었는지 (꼬리말/메뉴 제거) 확인
        if 'cleaned_text' in df.columns and len(df) >= 20:
            pd.set_option('display.max_colwidth', 500) # 텍스트가 잘리지 않게 설정
            print(df.sample(n=20)['cleaned_text'])
        elif 'cleaned_text' in df.columns:
            pd.set_option('display.max_colwidth', 500) # 텍스트가 잘리지 않게 설정
            print(df.sample(n=len(df))['cleaned_text'])
        else:
            print("'cleaned_text' 컬럼이 없거나 데이터가 3개 미만입니다.")
            
        print(f"\n--- 분석 완료 ---")

--- 'corpus.parquet' 파일 분석 시작 ---


[ 4. 정제된 텍스트(cleaned_text) 샘플 20개 ]
1437                                                                                                                                                                                                                                                                                                                                                                                                                                                                  April 03, 2024 Opening Remarks Chair Jerome H. Powell
1212                                                                                                                                                                                                                                                                                                                                                                                                                                

In [2]:
# ===============================================================
# STEP 1: FOMC JSONL 통합 → 통합 CSV 생성 (Python 3.9 호환)
# ===============================================================

import json, re, hashlib
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional
import pandas as pd

# 🔹 1. 폴더 설정 (필요시 수정)
BASE = Path("/statements/Data")  # 👉 네 JSONL 파일이 있는 폴더 경로

# 🔹 2. 입력 파일 목록
INPUTS = [
    ("fomc_statements_2000-present.jsonl", "statements"),
    ("fomc_minutes_2000-2019.jsonl", "minutes"),
    ("fomc_minutes_2020-present.jsonl", "minutes"),
    ("fomc_speeches_2000-present.jsonl", "speeches"),
]

# ===============================================================
# 헬퍼 함수들
# ===============================================================

def load_jsonl(path: Path) -> List[Dict[str, Any]]:
    """JSONL 파일을 안전하게 읽어서 리스트로 반환"""
    rows = []
    if not path.exists():
        print(f"[WARN] Missing file: {path.name}")
        return rows
    with path.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except Exception:
                try:
                    rows.append(json.loads(line.rstrip(",")))
                except Exception as e2:
                    rows.append({"__parse_error__": str(e2), "__raw__": line, "__line__": i})
    return rows


def normalize_whitespace(text: str) -> str:
    """공백·개행 정규화"""
    if not text:
        return ""
    text = text.replace("\xa0", " ").replace("\u200b", "")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s+\n", "\n", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def robust_parse_date(val: Any) -> Optional[datetime]:
    """다양한 날짜 포맷 자동 인식"""
    if val is None:
        return None
    if isinstance(val, (int, float)):
        try:
            return datetime.fromtimestamp(float(val))
        except Exception:
            return None
    if isinstance(val, str):
        s = val.strip()
        fmts = [
            "%Y-%m-%d", "%Y/%m/%d",
            "%b %d, %Y", "%B %d, %Y",
            "%d %b %Y", "%d %B %Y",
            "%Y-%m-%dT%H:%M:%S", "%Y-%m-%dT%H:%M:%SZ",
            "%m/%d/%Y"
        ]
        for fmt in fmts:
            try:
                return datetime.strptime(s, fmt)
            except Exception:
                pass
        # fallback: YYYY-MM-DD 패턴 추출
        m = re.match(r"(\d{4}-\d{2}-\d{2})", s)
        if m:
            try:
                return datetime.strptime(m.group(1), "%Y-%m-%d")
            except Exception:
                pass
    return None


def make_doc_id(text: str, date: Optional[datetime], source: str, title: str) -> str:
    """문서 고유 해시 ID 생성"""
    key = f"{source}|{title}|{date.isoformat() if isinstance(date, datetime) else ''}|{(text or '')[:2000]}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()[:16]

# ===============================================================
# 메인 처리
# ===============================================================

rows_out = []

for fname, source in INPUTS:
    path = BASE / fname
    data = load_jsonl(path)

    for r in data:
        # 파싱 실패 라인 스킵
        if "__parse_error__" in r and all(k not in r for k in ("text","content","body","raw_text","article","transcript")):
            continue

        # 키 통합
        text = r.get("text") or r.get("content") or r.get("body") or r.get("raw_text") or r.get("article") or r.get("transcript") or ""
        title = r.get("title") or r.get("heading") or r.get("name") or ""
        date_raw = r.get("date") or r.get("published") or r.get("time") or r.get("timestamp")
        url = r.get("url") or r.get("link") or r.get("source_url")
        speaker = r.get("speaker") or r.get("author") or r.get("presenter")
        meeting_id = r.get("meeting_id") or r.get("meetingId") or r.get("meeting")

        dt = robust_parse_date(date_raw)
        text_norm = normalize_whitespace(text)
        if not text_norm:
            continue

        if not meeting_id and isinstance(dt, datetime):
            meeting_id = dt.strftime("%Y%m%d")

        doc_id = make_doc_id(text_norm, dt, source, title or "")

        rows_out.append({
            "doc_id": doc_id,
            "source": source,
            "title": title or "",
            "date": dt.date().isoformat() if isinstance(dt, datetime) else None,
            "year": dt.year if isinstance(dt, datetime) else None,
            "url": url or "",
            "speaker": speaker or "",
            "meeting_id": str(meeting_id) if meeting_id else "",
            "text": text_norm,
            "char_len": len(text_norm),
        })

# ===============================================================
# CSV로 저장
# ===============================================================
unified = pd.DataFrame(rows_out).sort_values(["date","source","doc_id"], na_position="first").reset_index(drop=True)

unified_path = BASE / "unified_docs.csv"
manifest_path = BASE / "data_manifest.csv"

unified.to_csv(unified_path, index=False)
unified[["doc_id","source","date","year","title","url","speaker","meeting_id","char_len"]].to_csv(manifest_path, index=False)

print(f"[✅ 완료] {len(unified)}개의 문서를 통합 저장했습니다.")
print(f"- 통합 문서: {unified_path}")
print(f"- 메타 데이터: {manifest_path}")


[WARN] Missing file: fomc_statements_2000-present.jsonl
[WARN] Missing file: fomc_minutes_2000-2019.jsonl
[WARN] Missing file: fomc_minutes_2020-present.jsonl
[WARN] Missing file: fomc_speeches_2000-present.jsonl


KeyError: 'date'